In [1]:
import os
cur_path = os.getcwd()
engine_path = os.path.join(cur_path, 'OZEngine')
if engine_path not in sys.path:
    sys.path.append(engine_path)


In [2]:
from OZEngine.person_detectors import PersonDetector

detector = PersonDetector()

org_img = cv2.imread('image/navy_service_uniform/6.jpg', cv2.IMREAD_COLOR)
detector.detect(org_img)

dress checker
OCR loaded!


NameError: name 'net' is not defined